In [27]:
import pandas as pd
import openai
import requests
import json

In [28]:
# Load the CSV file with extracted grant information
input_csv_path = '/Users/miguelemb/Downloads/Projects/Grant scrapper/all_grants.csv'


In [29]:
# Limit the number of grants for testing
test_limit = 5

grants_df = pd.read_csv(input_csv_path).head(test_limit)


In [30]:
display(grants_df)

,OPPORTUNITY NUMBER,OPPORTUNITY TITLE,AGENCY CODE,OPPORTUNITY STATUS,POSTED DATE,CLOSE DATE,URL,General Information,Eligibility Information,Additional Information
0,"=HYPERLINK(""https://www.grants.gov/search-resu...",Regional Resilience Innovation Incubator,NSF,posted,08/09/2024,01/16/2025,https://www.grants.gov/search-results-detail/3...,"{'Document Type:': 'Grants Notice', 'Funding O...",{'Eligible Applicants:': 'Others (see text fie...,{'Agency Name:': 'National Science Foundation'...
1,"=HYPERLINK(""https://www.grants.gov/search-resu...",EPSCoR Research Incubators for STEM Excellence...,NSF,posted,05/13/2023,08/12/2025,https://www.grants.gov/search-results-detail/3...,This is for unrepresented minoriry founders. i...,"the gant amount is of $500,000.",Error extracting Additional Information: Messa...
2,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,"{'Document Type:': 'Grants Notice', 'Funding O...","{'Eligible Applicants:': 'Small businesses', '...",{'Agency Name:': 'National Institutes of Healt...
3,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,"{'Document Type:': 'Grants Notice', 'Funding O...","{'Eligible Applicants:': 'Small businesses', '...",{'Agency Name:': 'National Institutes of Healt...
4,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,"{'Document Type:': 'Grants Notice', 'Funding O...",{'Eligible Applicants:': 'City or township gov...,{'Agency Name:': 'National Institutes of Healt...


In [31]:
# Set up the OpenAI API key
openai.api_key = 'sk-NBbb1i54-gKgvS54YsbXmSpm2KhPKNHUyLBCyBsoj2T3BlbkFJli2IEFcxqOTg20t8EwELIWZGDLreEzoGne9G40MwMA'

In [32]:
founder_school_description = (
    "Founder School is a nonprofit organization focused on empowering underrepresented entrepreneurs across the United states,Caribbean, Central America, and South America, people of color, empower women entrepeneurs "
    "We provide resources, mentorship, and funding opportunities for individuals from marginalized communities who want to start or scale their businesses. "
    "Our primary focus is on promoting inclusive innovation and economic empowerment by providing access to networks, business education, technology training, and support. "
    "Founder School helps founders develop sustainable business models, refine their skills, and secure funding, particularly in underserved regions where traditional venture capital is less accessible. "
    "Our mission is to foster a supportive community where aspiring entrepreneurs can learn, connect, and grow, ultimately transforming their ideas into successful ventures."
)


In [33]:
# Define the columns that we will be adding to the DataFrame
columns = ['Apply Decision', 'Reason for Decision', 'Funding Amount']

In [34]:
# Ensure these columns are present in the original DataFrame
for col in columns:
    if col not in grants_df.columns:
        grants_df[col] = None


In [35]:
grants_df.head()

,OPPORTUNITY NUMBER,OPPORTUNITY TITLE,AGENCY CODE,OPPORTUNITY STATUS,POSTED DATE,CLOSE DATE,URL,General Information,Eligibility Information,Additional Information,Apply Decision,Reason for Decision,Funding Amount
0,"=HYPERLINK(""https://www.grants.gov/search-resu...",Regional Resilience Innovation Incubator,NSF,posted,08/09/2024,01/16/2025,https://www.grants.gov/search-results-detail/3...,"{'Document Type:': 'Grants Notice', 'Funding O...",{'Eligible Applicants:': 'Others (see text fie...,{'Agency Name:': 'National Science Foundation'...,None,None,None
1,"=HYPERLINK(""https://www.grants.gov/search-resu...",EPSCoR Research Incubators for STEM Excellence...,NSF,posted,05/13/2023,08/12/2025,https://www.grants.gov/search-results-detail/3...,This is for unrepresented minoriry founders. i...,"the gant amount is of $500,000.",Error extracting Additional Information: Messa...,None,None,None
2,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,"{'Document Type:': 'Grants Notice', 'Funding O...","{'Eligible Applicants:': 'Small businesses', '...",{'Agency Name:': 'National Institutes of Healt...,None,None,None
3,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,"{'Document Type:': 'Grants Notice', 'Funding O...","{'Eligible Applicants:': 'Small businesses', '...",{'Agency Name:': 'National Institutes of Healt...,None,None,None
4,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,"{'Document Type:': 'Grants Notice', 'Funding O...",{'Eligible Applicants:': 'City or township gov...,{'Agency Name:': 'National Institutes of Healt...,None,None,None


In [36]:
def analyze_grant(grant_info):
    # Create a prompt based on grant information
    prompt = (
        f"Given the following grant information, determine if it is a good fit for Founder School. "
        f"Founder School is described as follows: {founder_school_description}. "
        f"Provide a summary analysis of the grant's fit for Founder School in the following format: "
        f"'Decision: [Apply/Not Apply/Maybe] - Reason: [Reason for the decision] - Funding Amount: [Amount, if available]'. "
        f"Only provide an answer if you have all the necessary information to make a confident decision. If unsure, say 'Not Sure'."
        f"\nGrant Information:\n{grant_info}"
    )
    
    headers = {
        "Authorization": f"Bearer {openai.api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.5
    }
    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        headers=headers,
        json=data
    )
    response_json = response.json()
    return response_json['choices'][0]['message']['content'].strip()

In [37]:
for idx in range(len(grants_df)):
    grant_info = grants_df.iloc[idx].to_dict()
    gpt_analysis = analyze_grant(grant_info)
    
    # Split GPT analysis into decision, reason, and funding amount
    if "Decision:" in gpt_analysis and "Reason:" in gpt_analysis and "Funding Amount:" in gpt_analysis:
        try:
            decision_part = gpt_analysis.split("Decision:")[1].split("- Reason:")[0].strip()
            reason_part = gpt_analysis.split("Reason:")[1].split("- Funding Amount:")[0].strip()
            funding_amount_part = gpt_analysis.split("Funding Amount:")[1].strip()
            grants_df.at[idx, 'Apply Decision'] = decision_part
            grants_df.at[idx, 'Reason for Decision'] = reason_part
            grants_df.at[idx, 'Funding Amount'] = funding_amount_part
        except IndexError:
            grants_df.at[idx, 'Apply Decision'] = "Error parsing GPT response"
            grants_df.at[idx, 'Reason for Decision'] = "Error parsing GPT response"
            grants_df.at[idx, 'Funding Amount'] = "Error parsing GPT response"
    else:
        grants_df.at[idx, 'Apply Decision'] = "Error parsing GPT response"
        grants_df.at[idx, 'Reason for Decision'] = "Error parsing GPT response"
        grants_df.at[idx, 'Funding Amount'] = "Error parsing GPT response"



In [38]:
# Add a column to check if all columns were read before providing an answer
grants_df['All Columns Read'] = grants_df.apply(lambda row: 'Yes' if all(pd.notna(row)) else 'No', axis=1)

In [39]:
# Save the analyzed data to a new CSV file
grants_df.to_csv("gpt_analysis_results.csv", index=False)

In [40]:
display(grants_df)

,OPPORTUNITY NUMBER,OPPORTUNITY TITLE,AGENCY CODE,OPPORTUNITY STATUS,POSTED DATE,CLOSE DATE,URL,General Information,Eligibility Information,Additional Information,Apply Decision,Reason for Decision,Funding Amount,All Columns Read
0,"=HYPERLINK(""https://www.grants.gov/search-resu...",Regional Resilience Innovation Incubator,NSF,posted,08/09/2024,01/16/2025,https://www.grants.gov/search-results-detail/3...,"{'Document Type:': 'Grants Notice', 'Funding O...",{'Eligible Applicants:': 'Others (see text fie...,{'Agency Name:': 'National Science Foundation'...,Not Apply,The grant is focused on community-engaged team...,Not available.,Yes
1,"=HYPERLINK(""https://www.grants.gov/search-resu...",EPSCoR Research Incubators for STEM Excellence...,NSF,posted,05/13/2023,08/12/2025,https://www.grants.gov/search-results-detail/3...,This is for unrepresented minoriry founders. i...,"the gant amount is of $500,000.",Error extracting Additional Information: Messa...,Apply,The grant specifically targets underrepresente...,"$500,000.",Yes
2,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,"{'Document Type:': 'Grants Notice', 'Funding O...","{'Eligible Applicants:': 'Small businesses', '...",{'Agency Name:': 'National Institutes of Healt...,Not Apply,The grant is specifically aimed at small busin...,Not specified.,Yes
3,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,"{'Document Type:': 'Grants Notice', 'Funding O...","{'Eligible Applicants:': 'Small businesses', '...",{'Agency Name:': 'National Institutes of Healt...,Not Apply,The grant is specifically aimed at small busin...,Not available.,Yes
4,"=HYPERLINK(""https://www.grants.gov/search-resu...",Small Business Transition Grant for New Entrep...,HHS-NIH11,posted,04/25/2024,04/05/2027,https://www.grants.gov/search-results-detail/3...,"{'Document Type:': 'Grants Notice', 'Funding O...",{'Eligible Applicants:': 'City or township gov...,{'Agency Name:': 'National Institutes of Healt...,Not Apply,The grant is focused on supporting small busin...,Not available.,Yes
